# Toward Pattern Matching


## Java Releases
- Java 12: March 2019
  - switch expression (preview)
- Java 13: September 2019
  - switch expression (preview 2)
- Java 14: __March, 17th 2020__
  - switch expression (release)
  - record (preview)
  - instanceof (preview)


## Java Releases
- Java 12: March 2019
- Java 13: September 2019
- Java 14: March 2020
  - switch expression (release)
  - record (preview)
  - instanceof (preview)
- Java 15: __September, 15th 2020__
  - record (preview 2)
  - instanceof (preview 2)
  - selead type (preview)


## Features
- [switch expression](switch_expression.ipynb)
- [record](record.ipynb)
- [instanceof](instanceof.ipynb)
- [sealed types](sealed_types.ipynb)
